In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import pandas as pd
import numpy as np
import sqlite3
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os

import tkinter as tk
from tkinter import ttk, messagebox



class MRPSystem:
    def __init__(self, root,is_admin=True):
        self.root = root
        self.is_admin = is_admin
        self.root.title("Système MRP - Production de Skateboards")
        self.root.geometry("1200x800")
        self.root.configure(bg="#f0f0f0")
        print("MRPSystem window initialized")
        # Bind the close event to a custom handler
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

        # Initialisation de la base de données
        self.db_init()
        
        # Création des onglets
        self.tab_control = ttk.Notebook(root)
        
        # Onglet Accueil
        self.tab_home = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab_home, text='Accueil')
        
        # Onglet BOM (Nomenclature)
        self.tab_bom = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab_bom, text='Nomenclature (BOM)')
        
        # Onglet MRP
        self.tab_mrp = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab_mrp, text='Calcul MRP')
        
        # Onglet EOQ
        self.tab_eoq = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab_eoq, text='Calcul EOQ')
        
        # Onglet Rapport
        self.tab_report = ttk.Frame(self.tab_control)
        self.tab_control.add(self.tab_report, text='Rapports')
        
        self.tab_control.pack(expand=1, fill="both")
        
        # Initialisation des onglets
        self.setup_home_tab()
        self.setup_bom_tab()
        self.setup_mrp_tab()
        self.setup_eoq_tab()
        self.setup_report_tab()
        
        # Configuration du style
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("TButton", background="#0078D7", foreground="white", font=('Arial', 10, 'bold'))
        style.configure("TLabel", background="#f0f0f0", font=('Arial', 10))
        style.configure("TNotebook", background="#f0f0f0")
        style.map('TButton', background=[('active', '#005A9E')])
    def on_close(self):
        # Perform any necessary cleanup here
        if hasattr(self, 'conn'):
            self.conn.close()  # Close the database connection
        # Destroy the window
        self.root.destroy()
    def db_init(self):
        """Initialiser la base de données SQLite"""
        # Créer une base de données en mémoire pour l'exemple
        # Dans une application réelle, utilisez un fichier
        self.conn = sqlite3.connect(':memory:')
        self.cursor = self.conn.cursor()
        
        # Créer les tables
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS items (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            lead_time INTEGER,
            on_hand INTEGER,
            lot_size TEXT,
            is_purchased BOOLEAN,
            unit_cost REAL,
            holding_cost REAL,
            ordering_cost REAL
        )
        ''')
        
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS bom (
            parent_id INTEGER,
            child_id INTEGER,
            quantity INTEGER,
            PRIMARY KEY (parent_id, child_id),
            FOREIGN KEY (parent_id) REFERENCES items (id),
            FOREIGN KEY (child_id) REFERENCES items (id)
        )
        ''')
        
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS mps (
            period INTEGER,
            item_id INTEGER,
            quantity INTEGER,
            PRIMARY KEY (period, item_id),
            FOREIGN KEY (item_id) REFERENCES items (id)
        )
        ''')
        
        # Données initiales pour les articles
        items = [
            (1, 'Skateboard', 2, 650, 'Lot-for-lot', 0, 0, 0, 0),
            (2, 'Board', 1, 550, 'Lot-for-lot', 0, 0, 0, 0),
            (3, 'Trucks', 2, 15, 'Multiples of 100', 1, 20, 5, 65),
            (4, 'Wheels', 3, 120, 'Multiples of 40', 1, 0, 0, 0),
            (5, 'Screws', 2, 0, 'Multiples of 100', 1, 0, 0, 0),
            (6, 'Tire', 1, 150, 'Multiples of 50', 1, 0, 0, 0),
            (7, 'Rim', 1, 200, 'Multiples of 20', 1, 0, 0, 0)
        ]
        
        self.cursor.executemany('INSERT OR REPLACE INTO items VALUES (?,?,?,?,?,?,?,?,?)', items)
        
        # Données initiales pour la nomenclature (BOM)
        bom_data = [
            (1, 2, 1),  # 1 Skateboard nécessite 1 Board
            (1, 3, 2),  # 1 Skateboard nécessite 2 Trucks
            (1, 4, 4),  # 1 Skateboard nécessite 4 Wheels
            (2, 5, 8),  # 1 Board nécessite 8 Screws
            (4, 6, 1),  # 1 Wheel nécessite 1 Tire
            (4, 7, 1),  # 1 Wheel nécessite 1 Rim
            (4, 5, 4)   # 1 Wheel nécessite 4 Screws
        ]
        
        self.cursor.executemany('INSERT OR REPLACE INTO bom VALUES (?,?,?)', bom_data)
        
        # Table MPS vide au départ (sera remplie par l'utilisateur)
        self.cursor.execute("DELETE FROM mps WHERE item_id = 1")  # Skateboard
        
        self.conn.commit()

    def setup_home_tab(self):
        """Configuration de l'onglet Accueil"""
        frame = ttk.Frame(self.tab_home, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # Logo et titre
        title_label = ttk.Label(frame, text="SYSTÈME MRP", font=('Arial', 24, 'bold'))
        title_label.pack(pady=20)
        
        subtitle_label = ttk.Label(frame, text="Gestion de la planification de production", font=('Arial', 16))
        subtitle_label.pack(pady=10)
        
        # Information sur le système
        info_frame = ttk.LabelFrame(frame, text="À propos du système", padding=10)
        info_frame.pack(fill=tk.X, pady=20)
        
        # Ajouter un bouton de sortie
        exit_button = ttk.Button(frame, text="Quitter", command=self.root.destroy)
        exit_button.pack(pady=10)

        info_text = """Ce système permet de gérer la planification de production de skateboards.
        
Fonctionnalités principales:
- Gestion de la nomenclature (BOM)
- Calcul des besoins matières (MRP)
- Calcul des quantités économiques de commande (EOQ)
- Génération de rapports

Pour commencer, sélectionnez un onglet ci-dessus.
        """
        
        info_label = ttk.Label(info_frame, text=info_text, justify=tk.LEFT)
        info_label.pack(pady=10)
        
        # Raccourcis vers les fonctionnalités principales
        shortcuts_frame = ttk.Frame(frame)
        shortcuts_frame.pack(fill=tk.X, pady=20)
        
        btn_bom = ttk.Button(shortcuts_frame, text="Consulter la nomenclature",
                          command=lambda: self.tab_control.select(1))
        btn_bom.grid(row=0, column=0, padx=10, pady=10)
        
        btn_mrp = ttk.Button(shortcuts_frame, text="Lancer calcul MRP",
                           command=lambda: self.tab_control.select(2))
        btn_mrp.grid(row=0, column=1, padx=10, pady=10)
        
        btn_eoq = ttk.Button(shortcuts_frame, text="Calculer EOQ",
                           command=lambda: self.tab_control.select(3))
        btn_eoq.grid(row=0, column=2, padx=10, pady=10)
        
        btn_report = ttk.Button(shortcuts_frame, text="Générer rapports",
                             command=lambda: self.tab_control.select(4))
        btn_report.grid(row=0, column=3, padx=10, pady=10)

    def setup_bom_tab(self):
        """Configuration de l'onglet BOM (nomenclature)"""
        frame = ttk.Frame(self.tab_bom, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # Titre
        title_label = ttk.Label(frame, text="Nomenclature (BOM)", font=('Arial', 16, 'bold'))
        title_label.pack(pady=10)
        
        # Frame pour la vue hiérarchique
        tree_frame = ttk.LabelFrame(frame, text="Structure du produit", padding=10)
        tree_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Treeview pour afficher la structure BOM hiérarchique
        self.bom_tree = ttk.Treeview(tree_frame, columns=("qty", "lead_time", "on_hand", "lot_size"))
        self.bom_tree.heading("#0", text="Composant")
        self.bom_tree.heading("qty", text="Quantité")
        self.bom_tree.heading("lead_time", text="Délai (semaines)")
        self.bom_tree.heading("on_hand", text="Stock disponible")
        self.bom_tree.heading("lot_size", text="Règle de lot")
        
        self.bom_tree.column("#0", width=250)
        self.bom_tree.column("qty", width=80, anchor=tk.CENTER)
        self.bom_tree.column("lead_time", width=120, anchor=tk.CENTER)
        self.bom_tree.column("on_hand", width=120, anchor=tk.CENTER)
        self.bom_tree.column("lot_size", width=120, anchor=tk.CENTER)
        
        self.bom_tree.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Scrollbar pour le Treeview
        scrollbar = ttk.Scrollbar(tree_frame, orient="vertical", command=self.bom_tree.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.bom_tree.configure(yscrollcommand=scrollbar.set)
        
        # Boutons pour gérer la nomenclature
        btn_frame = ttk.Frame(frame)
        btn_frame.pack(fill=tk.X, pady=10)
        
        refresh_btn = ttk.Button(btn_frame, text="Rafraîchir", command=self.refresh_bom_tree)
        refresh_btn.pack(side=tk.LEFT, padx=5)
        
        add_btn = ttk.Button(btn_frame, text="Ajouter composant", command=self.add_component)
        add_btn.pack(side=tk.LEFT, padx=5)
        
        edit_btn = ttk.Button(btn_frame, text="Modifier composant", command=self.edit_component)
        edit_btn.pack(side=tk.LEFT, padx=5)
        
        # Charger les données initiales
        self.refresh_bom_tree()

    def configure_requirements(self):
        """Ouvre l'interface de configuration des besoins (admin seulement)"""
        if not self.is_admin:
            self.view_requirements()
            return
        
        periods = self.periods_var.get()
        
        config_window = tk.Toplevel(self.root)
        config_window.title("Configuration des besoins - Mode Admin")
        config_window.geometry("500x400")
        
        # Frame avec scrollbar
        canvas = tk.Canvas(config_window)
        scrollbar = ttk.Scrollbar(config_window, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Titre
        ttk.Label(scrollable_frame, text="Configuration des besoins par période", 
                 font=('Arial', 12, 'bold')).pack(pady=10)
        
        # Récupérer les valeurs existantes
        self.cursor.execute("SELECT period, quantity FROM mps WHERE item_id = 1 ORDER BY period")
        existing_data = {period: qty for period, qty in self.cursor.fetchall()}
        
        # Tableau de saisie
        self.requirement_entries = []
        for i in range(1, periods + 1):
            frame = ttk.Frame(scrollable_frame)
            frame.pack(fill=tk.X, pady=2, padx=10)
            
            ttk.Label(frame, text=f"Période {i}:").pack(side=tk.LEFT, padx=5)
            
            var = tk.IntVar(value=existing_data.get(i, 0))
            entry = ttk.Entry(frame, textvariable=var, width=10)
            entry.pack(side=tk.LEFT, padx=5)
            
            self.requirement_entries.append((i, var))
        
        # Boutons
        btn_frame = ttk.Frame(scrollable_frame)
        btn_frame.pack(pady=15)
        
        ttk.Button(btn_frame, text="Enregistrer", command=lambda: self.save_requirements(config_window)).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="Annuler", command=config_window.destroy).pack(side=tk.LEFT, padx=5)

    def view_requirements(self):
        """Affiche les besoins en lecture seule (mode utilisateur)"""
        periods = self.periods_var.get()
        
        view_window = tk.Toplevel(self.root)
        view_window.title("Visualisation des besoins")
        view_window.geometry("300x400")
        
        # Récupérer les données
        self.cursor.execute("SELECT period, quantity FROM mps WHERE item_id = 1 ORDER BY period")
        requirements = {period: qty for period, qty in self.cursor.fetchall()}
        
        # Treeview pour affichage
        tree_frame = ttk.Frame(view_window)
        tree_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        tree = ttk.Treeview(tree_frame, columns=("period", "quantity"), show="headings", height=15)
        tree.heading("period", text="Période")
        tree.heading("quantity", text="Quantité")
        tree.column("period", width=120)
        tree.column("quantity", width=120, anchor=tk.CENTER)
        
        vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=vsb.set)
        
        tree.grid(row=0, column=0, sticky="nsew")
        vsb.grid(row=0, column=1, sticky="ns")
        
        for i in range(1, periods + 1):
            qty = requirements.get(i, 0)
            tree.insert("", "end", values=(f"Période {i}", qty))
        
        # Bouton Fermer
        ttk.Button(view_window, text="Fermer", command=view_window.destroy).pack(pady=10)
    
    def save_requirements(self, window):
        """Sauvegarde les besoins en base de données"""
        try:
            self.cursor.execute("DELETE FROM mps WHERE item_id = 1")
            
            mps_data = []
            for period, var in self.requirement_entries:
                quantity = var.get()
                if quantity > 0:
                    mps_data.append((period, 1, quantity))
            
            self.cursor.executemany("INSERT INTO mps VALUES (?,?,?)", mps_data)
            self.conn.commit()
            window.destroy()
            messagebox.showinfo("Succès", "Configuration sauvegardée!")
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la sauvegarde:\n{str(e)}")
            
    def setup_mrp_tab(self):
        """Configuration de l'onglet MRP avec gestion des permissions"""
        frame = ttk.Frame(self.tab_mrp, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # Titre
        title_label = ttk.Label(frame, text="Calcul MRP", font=('Arial', 16, 'bold'))
        title_label.pack(pady=10)
        
        # Cadre pour les paramètres
        param_frame = ttk.LabelFrame(frame, text="Paramètres de calcul", padding=10)
        param_frame.pack(fill=tk.X, pady=10)
        
        # Nombre de périodes
        ttk.Label(param_frame, text="Nombre de périodes:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.periods_var = tk.IntVar(value=16)
        self.periods_entry = ttk.Spinbox(param_frame, from_=1, to=52, textvariable=self.periods_var, width=5)
        self.periods_entry.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        
        # Produit à calculer (fixé sur Skateboard)
        ttk.Label(param_frame, text="Produit:").grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        self.product_var = tk.StringVar(value="Skateboard")
        self.product_combo = ttk.Combobox(param_frame, textvariable=self.product_var, 
                                        values=["Skateboard"], width=15, state="readonly")
        self.product_combo.grid(row=0, column=3, padx=5, pady=5, sticky=tk.W)
        
        # Bouton pour configurer les besoins
        self.config_btn = ttk.Button(param_frame, text="Configurer besoins", 
                                   command=self.configure_requirements)
        self.config_btn.grid(row=0, column=4, padx=5, pady=5)
        
        # Bouton de calcul
        self.calculate_btn = ttk.Button(param_frame, text="Calculer MRP", command=self.calculate_mrp)
        self.calculate_btn.grid(row=0, column=5, padx=5, pady=5)
        
        # Cadre pour les résultats MRP
        self.mrp_result_frame = ttk.LabelFrame(frame, text="Résultats MRP", padding=10)
        self.mrp_result_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Notebook pour les différents composants
        self.mrp_notebook = ttk.Notebook(self.mrp_result_frame)
        self.mrp_notebook.pack(fill=tk.BOTH, expand=True)
        
        # Appliquer les permissions
        if not self.is_admin:
            self.config_btn.config(state=tk.DISABLED)
            self.periods_entry.config(state=tk.DISABLED)
        
    def setup_admin_permissions(self):
        """Configure les permissions selon le rôle"""
        if not self.is_admin:
            # Désactiver les onglets de modification
            self.tab_control.tab(1, state="disabled")  # Onglet BOM
            self.tab_control.tab(3, state="disabled")  # Onglet EOQ
            
            # Champs en lecture seule
            for widget in [self.periods_entry, self.product_combo]:
                widget.config(state="readonly")
            
            # Style visuel différent pour le mode utilisateur
            style = ttk.Style()
            style.configure("User.TLabel", background="#f0f0f0", foreground="#555555")
            style.configure("User.TButton", background="#e0e0e0", foreground="#333333")
            
            # Appliquer le style aux éléments concernés
            for tab in [self.tab_home, self.tab_mrp, self.tab_report]:
                for child in tab.winfo_children():
                    if isinstance(child, (ttk.Label, ttk.Button)):
                        child.configure(style="User.TLabel" if isinstance(child, ttk.Label) else "User.TButton")
        
    def setup_eoq_tab(self):
        """Configuration de l'onglet EOQ"""
        frame = ttk.Frame(self.tab_eoq, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # Titre
        title_label = ttk.Label(frame, text="Calcul des Quantités Économiques de Commande (EOQ)", 
                              font=('Arial', 16, 'bold'))
        title_label.pack(pady=10)
        
        # Frame pour les paramètres d'EOQ
        param_frame = ttk.LabelFrame(frame, text="Paramètres de calcul", padding=10)
        param_frame.pack(fill=tk.X, pady=10)
        
        # Sélection du composant
        ttk.Label(param_frame, text="Sélectionner un composant:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.eoq_component_var = tk.StringVar()
        
        # Récupérer les composants achetés
        self.cursor.execute("SELECT name FROM items WHERE is_purchased = 1")
        purchased_items = [row[0] for row in self.cursor.fetchall()]
        
        component_combo = ttk.Combobox(param_frame, textvariable=self.eoq_component_var, 
                                      values=purchased_items, width=15)
        component_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        
        # Demande annuelle
        ttk.Label(param_frame, text="Demande annuelle (D):").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        self.annual_demand_var = tk.IntVar(value=60000)
        annual_demand_entry = ttk.Entry(param_frame, textvariable=self.annual_demand_var, width=10)
        annual_demand_entry.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)
        
        # Coût unitaire
        ttk.Label(param_frame, text="Coût unitaire ($):").grid(row=1, column=2, padx=5, pady=5, sticky=tk.W)
        self.unit_cost_var = tk.DoubleVar(value=20.0)
        unit_cost_entry = ttk.Entry(param_frame, textvariable=self.unit_cost_var, width=10)
        unit_cost_entry.grid(row=1, column=3, padx=5, pady=5, sticky=tk.W)
        
        # Coût de possession
        ttk.Label(param_frame, text="Coût de possession ($/unité/an):").grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
        self.holding_cost_var = tk.DoubleVar(value=5.0)
        holding_cost_entry = ttk.Entry(param_frame, textvariable=self.holding_cost_var, width=10)
        holding_cost_entry.grid(row=2, column=1, padx=5, pady=5, sticky=tk.W)
        
        # Coût de commande
        ttk.Label(param_frame, text="Coût de commande ($):").grid(row=2, column=2, padx=5, pady=5, sticky=tk.W)
        self.ordering_cost_var = tk.DoubleVar(value=65.0)
        ordering_cost_entry = ttk.Entry(param_frame, textvariable=self.ordering_cost_var, width=10)
        ordering_cost_entry.grid(row=2, column=3, padx=5, pady=5, sticky=tk.W)
        
        # Jours ouvrés par an
        ttk.Label(param_frame, text="Jours ouvrés par an:").grid(row=3, column=0, padx=5, pady=5, sticky=tk.W)
        self.working_days_var = tk.IntVar(value=200)
        working_days_entry = ttk.Entry(param_frame, textvariable=self.working_days_var, width=10)
        working_days_entry.grid(row=3, column=1, padx=5, pady=5, sticky=tk.W)
        
        # Bouton de calcul
        calculate_btn = ttk.Button(param_frame, text="Calculer EOQ", command=self.calculate_eoq)
        calculate_btn.grid(row=3, column=2, columnspan=2, padx=5, pady=10)
        
        # Frame pour les résultats
        self.eoq_result_frame = ttk.LabelFrame(frame, text="Résultats EOQ", padding=10)
        self.eoq_result_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Graphique pour afficher le coût total
        self.fig, self.ax = plt.subplots(figsize=(8, 4))
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.eoq_result_frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Zone de texte pour les résultats
        self.eoq_result_text = tk.Text(self.eoq_result_frame, height=6, width=80)
        self.eoq_result_text.pack(fill=tk.X, expand=False, pady=10)
        
    def setup_report_tab(self):
        """Configuration de l'onglet Rapport"""
        frame = ttk.Frame(self.tab_report, padding=20)
        frame.pack(fill=tk.BOTH, expand=True)
        
        # Titre
        title_label = ttk.Label(frame, text="Rapports", font=('Arial', 16, 'bold'))
        title_label.pack(pady=10)
        
        # Options de rapport
        options_frame = ttk.LabelFrame(frame, text="Options de rapport", padding=10)
        options_frame.pack(fill=tk.X, pady=10)
        
        # Type de rapport
        ttk.Label(options_frame, text="Type de rapport:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        self.report_type_var = tk.StringVar(value="Inventaire")
        report_types = ["Inventaire", "Planification des commandes", "Besoins nets", "Résumé EOQ"]
        report_type_combo = ttk.Combobox(options_frame, textvariable=self.report_type_var, 
                                        values=report_types, width=25)
        report_type_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
        
        # Format de sortie
        ttk.Label(options_frame, text="Format de sortie:").grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
        self.output_format_var = tk.StringVar(value="Afficher")
        output_formats = ["Afficher", "CSV", "Excel"]
        output_format_combo = ttk.Combobox(options_frame, textvariable=self.output_format_var, 
                                          values=output_formats, width=15)
        output_format_combo.grid(row=0, column=3, padx=5, pady=5, sticky=tk.W)
        
        # Bouton de génération
        generate_btn = ttk.Button(options_frame, text="Générer rapport", command=self.generate_report)
        generate_btn.grid(row=0, column=4, padx=20, pady=5)
        
        # Zone pour afficher le rapport
        self.report_frame = ttk.LabelFrame(frame, text="Rapport", padding=10)
        self.report_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Nous utiliserons un Treeview pour afficher les données du rapport
        self.report_tree = ttk.Treeview(self.report_frame)
        self.report_tree.pack(fill=tk.BOTH, expand=True, side=tk.LEFT)
        
        # Scrollbar pour le Treeview
        vsb = ttk.Scrollbar(self.report_frame, orient="vertical", command=self.report_tree.yview)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        self.report_tree.configure(yscrollcommand=vsb.set)
        
    def refresh_bom_tree(self):
        """Rafraîchir l'arbre de la nomenclature"""
        # Effacer l'arbre actuel
        for item in self.bom_tree.get_children():
            self.bom_tree.delete(item)
        
        # Récupérer le produit principal (skateboard)
        self.cursor.execute("SELECT id, name, lead_time, on_hand, lot_size FROM items WHERE name = 'Skateboard'")
        skateboard = self.cursor.fetchone()
        
        if skateboard:
            # Ajouter le skateboard comme racine
            root_id = self.bom_tree.insert("", "end", text=skateboard[1], values=("", skateboard[2], 
                                                                                skateboard[3], skateboard[4]))
            
            # Ajouter les enfants récursivement
            self.add_children_to_tree(root_id, skateboard[0])
    
    def add_children_to_tree(self, parent_tree_id, parent_item_id):
        """Ajouter récursivement les enfants à l'arbre"""
        # Récupérer les composants enfants
        query = """
        SELECT i.id, i.name, b.quantity, i.lead_time, i.on_hand, i.lot_size
        FROM bom b
        JOIN items i ON b.child_id = i.id
        WHERE b.parent_id = ?
        """
        self.cursor.execute(query, (parent_item_id,))
        children = self.cursor.fetchall()
        
        for child in children:
            child_id, name, qty, lead_time, on_hand, lot_size = child
            child_tree_id = self.bom_tree.insert(parent_tree_id, "end", text=name, 
                                               values=(qty, lead_time, on_hand, lot_size))
            
            # Vérifier si ce composant a lui-même des enfants
            self.cursor.execute("SELECT COUNT(*) FROM bom WHERE parent_id = ?", (child_id,))
            has_children = self.cursor.fetchone()[0] > 0
            
            if has_children:
                self.add_children_to_tree(child_tree_id, child_id)
    
    def add_component(self):
        """Ajouter un nouveau composant"""
        # Cette fonction pourrait être développée davantage avec une fenêtre de dialogue
        messagebox.showinfo("Information", "Fonctionnalité à implémenter")
    
    def edit_component(self):
        """Modifier un composant existant"""
        # Cette fonction pourrait être développée davantage avec une fenêtre de dialogue
        messagebox.showinfo("Information", "Fonctionnalité à implémenter")
    
    def calculate_mrp(self):
        """Lance le calcul MRP avec gestion des erreurs"""
        try:
            product_name = self.product_var.get()
            periods = self.periods_var.get()
            
            # Vérifier la présence de données
            self.cursor.execute("SELECT COUNT(*) FROM mps WHERE item_id = 1")
            if self.cursor.fetchone()[0] == 0:
                messagebox.showwarning("Attention", "Aucun besoin configuré!\nConfigurez les besoins d'abord.")
                return
            
            # Effacer les résultats précédents
            for tab in self.mrp_notebook.tabs():
                self.mrp_notebook.forget(tab)
            
            # Récupérer l'ID du produit
            self.cursor.execute("SELECT id FROM items WHERE name = ?", (product_name,))
            product_id = self.cursor.fetchone()[0]
            
            # Calculer le MRP
            components = self.get_all_components(product_id)
            self.planned_releases = {comp_id: [0]*(periods+1) for comp_id, _ in components}
            
            for component_id, component_name in components:
                self.calculate_component_mrp(component_id, component_name, periods)
            
            messagebox.showinfo("Calcul terminé", "Le plan MRP a été généré avec succès!")
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Une erreur est survenue:\n{str(e)}")

    def calculate_component_mrp(self, product_id, periods):
        """Calculer le MRP pour tous les composants d'un produit"""
        # Récupérer tous les composants connectés au produit
        components = self.get_all_components(product_id)
        
        # Pour chaque composant, calculer le MRP
        for component_id, component_name in components:
            # Créer un nouvel onglet pour ce composant
            component_frame = ttk.Frame(self.mrp_notebook)
            self.mrp_notebook.add(component_frame, text=component_name)
            
            # Récupérer les données de l'item
            self.cursor.execute("""
            SELECT lead_time, on_hand, lot_size
            FROM items WHERE id = ?
            """, (component_id,))
            lead_time, on_hand, lot_size_rule = self.cursor.fetchone()
            
            # Créer et configurer le tableau MRP
            mrp_tree = ttk.Treeview(component_frame, columns=[f"p{i}" for i in range(periods+1)])
            mrp_tree.heading("#0", text="MRP")
            
            for i in range(periods+1):
                mrp_tree.heading(f"p{i}", text=f"P{i}")
                mrp_tree.column(f"p{i}", width=60, anchor=tk.CENTER)
            
            mrp_tree.column("#0", width=150)
            
            # Ajouter les lignes du MRP
            rows = ["Besoins bruts", "Réceptions prévues", "Stock prévisionnel", 
                   "Besoins nets", "Réceptions planifiées", "Lancements planifiés"]
            
            for row in rows:
                mrp_tree.insert("", "end", text=row, values=["" for _ in range(periods+1)])
            
            mrp_tree.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
            
            # Dans une application réelle, calculer les valeurs MRP ici
            
            # Pour l'exemple, nous allons calculer quelques valeurs fictives
            self.calculate_demo_mrp(mrp_tree, component_id, lead_time, on_hand, lot_size_rule, periods)
    
    def calculate_demo_mrp(self, tree, component_id, lead_time, on_hand, lot_size_rule, periods):
        """Calculer des valeurs démonstratives de MRP"""
        # Récupérer les enfants du tree (lignes MRP)
        items = tree.get_children()
        gross_req_id = items[0]  # Besoins bruts
        scheduled_receipts_id = items[1]  # Réceptions prévues
        projected_on_hand_id = items[2]  # Stock prévisionnel
        net_req_id = items[3]  # Besoins nets
        planned_receipts_id = items[4]  # Réceptions planifiées
        planned_releases_id = items[5]  # Lancements planifiés
        
        # Initialize arrays
        gross_reqs = [0] * (periods + 1)
        scheduled_receipts = [0] * (periods + 1)
        projected_oh = [0] * (periods + 1)
        net_reqs = [0] * (periods + 1)
        planned_recs = [0] * (periods + 1)
        planned_rels = [0] * (periods + 1)
        
        # Stock initial
        projected_oh[0] = on_hand
        
        # Pour le skateboard, récupérer les besoins bruts depuis le MPS
        if component_id == 1:  # Si c'est le skateboard
            self.cursor.execute("SELECT period, quantity FROM mps WHERE item_id = ? ORDER BY period", (component_id,))
            for period, qty in self.cursor.fetchall():
                if period <= periods:
                    gross_reqs[period] = qty
        else:
            # Pour les autres composants, calculer les besoins bruts basés sur les lancements planifiés du parent
            # Trouver tous les parents de ce composant et leurs quantités dans le BOM
            self.cursor.execute("""
                SELECT b.parent_id, b.quantity 
                FROM bom b 
                WHERE b.child_id = ?
            """, (component_id,))
            parent_data = self.cursor.fetchall()
            
            for parent_id, quantity in parent_data:
                # Trouver les lancements planifiés du parent
                parent_planned_releases = self.get_planned_releases(parent_id, periods)
                
                # Calculer les besoins bruts pour ce composant
                for period in range(periods + 1):
                    gross_reqs[period] += parent_planned_releases[period] * quantity
        
        tree.item(gross_req_id, values=gross_reqs)
        
        # Réceptions prévues (Scheduled Receipts)
        # Dans une vraie application, vous récupéreriez les commandes déjà passées
        tree.item(scheduled_receipts_id, values=scheduled_receipts)
        
        # Calcul du MRP
        for i in range(1, periods + 1):
            # Étape 1 : Stock initial disponible
            stock_avant_reception = projected_oh[i-1]
            
            # Étape 2 : Ajouter les réceptions prévues au stock
            stock_disponible = stock_avant_reception + scheduled_receipts[i]

            # Étape 3 : Besoins à couvrir
            demande_totale = gross_reqs[i]

            SAFETY_STOCK = 0  # Changer à une valeur > 0 si nécessaire

            # Étape 4 : Consommer le stock et les réceptions prévues immédiatement
            if stock_disponible - demande_totale > SAFETY_STOCK:
                projected_oh[i] = stock_disponible - demande_totale
                net_reqs[i] = 0  # Pas de besoin net
            else:
                projected_oh[i] = max(0, stock_disponible - demande_totale)  # Évite les valeurs négatives
                net_reqs[i] = max(0, demande_totale - stock_disponible + SAFETY_STOCK)  # Ajoute un besoin net si stock < seuil

            # Étape 5 : Planifier une commande si besoin
            if net_reqs[i] > 0:
                if "Lot-for-lot" in lot_size_rule:
                    lot_size = net_reqs[i]
                elif "Multiples of" in lot_size_rule:
                    lot_multiple = int(lot_size_rule.split()[-1])
                    lot_size = math.ceil(net_reqs[i] / lot_multiple) * lot_multiple
                else:
                    lot_size = net_reqs[i]
                
                planned_recs[i] = lot_size
                
                # Planifier le lancement avec le lead time
                if i - lead_time >= 0:
                    planned_rels[i - lead_time] = lot_size
        
        # Mettre à jour l'arbre
        tree.item(projected_on_hand_id, values=projected_oh)
        tree.item(net_req_id, values=net_reqs)
        tree.item(planned_receipts_id, values=planned_recs)
        tree.item(planned_releases_id, values=planned_rels)


    
    def get_all_components(self, product_id):
        """Récupérer tous les composants dans l'ordre hiérarchique correct (top-down)"""
        components = []
        # Start with the product itself
        self.cursor.execute("SELECT id, name FROM items WHERE id = ?", (product_id,))
        product = self.cursor.fetchone()
        if product:
            components.append(product)
        
        # Then get components in BFS order (top-down)
        queue = [product_id]
        while queue:
            parent_id = queue.pop(0)
            # Get direct components
            self.cursor.execute("""
            SELECT i.id, i.name
            FROM bom b
            JOIN items i ON b.child_id = i.id
            WHERE b.parent_id = ?
            ORDER BY i.id
            """, (parent_id,))
            children = self.cursor.fetchall()
            
            for child_id, child_name in children:
                if (child_id, child_name) not in components:
                    components.append((child_id, child_name))
                    queue.append(child_id)
        
        return (components)

    def calculate_mrp(self):
        """Calculer le MRP pour le produit sélectionné"""
        periods = self.periods_var.get()
        
        # Effacer les onglets existants
        for tab in self.mrp_notebook.tabs():
            self.mrp_notebook.forget(tab)
        
        # ID du skateboard (produit final)
        product_id = 1
        
        # Get components in correct order (top-down)
        components = self.get_all_components(product_id)
        
        # Initialisation des lancements planifiés
        self.planned_releases = {comp_id: [0]*(periods+1) for comp_id, _ in components}
        
        # Calculate MRP for each component in order
        for component_id, component_name in components:
            self.calculate_component_mrp(component_id, component_name, periods)
        
        messagebox.showinfo("Information", "Calcul MRP terminé!")
            


    def calculate_component_mrp(self, component_id, component_name, periods):
            """Calculer le MRP pour un composant spécifique"""
            # Créer un nouvel onglet pour ce composant
            component_frame = ttk.Frame(self.mrp_notebook)
            self.mrp_notebook.add(component_frame, text=component_name)
            
            # Récupérer les données de l'item
            self.cursor.execute("""
            SELECT lead_time, on_hand, lot_size
            FROM items WHERE id = ?
            """, (component_id,))
            lead_time, on_hand, lot_size_rule = self.cursor.fetchone()
            
            # Créer et configurer le tableau MRP
            mrp_tree = ttk.Treeview(component_frame, columns=[f"p{i}" for i in range(periods+1)])
            mrp_tree.heading("#0", text="MRP")
            
            for i in range(periods+1):
                mrp_tree.heading(f"p{i}", text=f"P{i}")
                mrp_tree.column(f"p{i}", width=60, anchor=tk.CENTER)
            
            mrp_tree.column("#0", width=150)
            
            # Ajouter les lignes du MRP
            rows = ["Besoins bruts", "Réceptions prévues", "Stock prévisionnel", 
                "Besoins nets", "Réceptions planifiées", "Lancements planifiés"]
            
            for row in rows:
                mrp_tree.insert("", "end", text=row, values=["" for _ in range(periods+1)])
            
            mrp_tree.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
            
            # Calculer les valeurs MRP
            self.calculate_mrp_values(mrp_tree, component_id, lead_time, on_hand, lot_size_rule, periods)

    def calculate_mrp_values(self, tree, component_id, lead_time, on_hand, lot_size_rule, periods):
        """Calculer les valeurs MRP pour un composant"""
        items = tree.get_children()
        gross_req_id = items[0]
        scheduled_receipts_id = items[1]
        projected_on_hand_id = items[2]
        net_req_id = items[3]
        planned_receipts_id = items[4]
        planned_releases_id = items[5]
        
        # Initialize arrays
        gross_reqs = [0] * (periods + 1)
        scheduled_receipts = [0] * (periods + 1)
        projected_oh = [0] * (periods + 1)
        net_reqs = [0] * (periods + 1)
        planned_recs = [0] * (periods + 1)
        planned_rels = [0] * (periods + 1)
        
        # Stock initial
        projected_oh[0] = on_hand
        
        # Pour le skateboard, utiliser le MPS
        if component_id == 1:
            self.cursor.execute("SELECT period, quantity FROM mps WHERE item_id = ? ORDER BY period", (component_id,))
            for period, qty in self.cursor.fetchall():
                if period <= periods:
                    gross_reqs[period] = qty
        else:
            # CAS SPECIAL POUR LES SCREWS (ID=5)
            if component_id == 5:
                # Besoins venant du Board (8 par board)
                self.cursor.execute("""
                    SELECT b.parent_id, b.quantity 
                    FROM bom b 
                    WHERE b.child_id = ? AND b.parent_id = 2
                """, (component_id,))
                parent_data_board = self.cursor.fetchall()
                
                # Besoins venant des Wheels (4 par wheel)
                self.cursor.execute("""
                    SELECT b.parent_id, b.quantity 
                    FROM bom b 
                    WHERE b.child_id = ? AND b.parent_id = 4
                """, (component_id,))
                parent_data_wheels = self.cursor.fetchall()
                
                # Somme des deux sources
                for parent_id, quantity in parent_data_board + parent_data_wheels:
                    parent_planned_releases = self.planned_releases.get(parent_id, [0]*(periods+1))
                    for period in range(periods + 1):
                        gross_reqs[period] += parent_planned_releases[period] * quantity
            else:
                # Calcul normal pour les autres composants
                self.cursor.execute("SELECT b.parent_id, b.quantity FROM bom b WHERE b.child_id = ?", (component_id,))
                parent_data = self.cursor.fetchall()
                
                for parent_id, quantity in parent_data:
                    parent_planned_releases = self.planned_releases.get(parent_id, [0]*(periods+1))
                    for period in range(periods + 1):
                        gross_reqs[period] += parent_planned_releases[period] * quantity
        
        # Calcul du MRP
        for i in range(1, periods + 1):
            # Stock disponible = stock précédent + réceptions prévues
            stock_disponible = projected_oh[i-1] + scheduled_receipts[i]
            
            # Calcul du besoin net
            besoin_net = max(0, gross_reqs[i] - stock_disponible)
            
            if besoin_net > 0:
                # Appliquer la règle de lot
                if "Lot-for-lot" in lot_size_rule:
                    lot_size = besoin_net
                elif "Multiples of" in lot_size_rule:
                    multiple = int(lot_size_rule.split()[-1])
                    lot_size = math.ceil(besoin_net / multiple) * multiple
                else:
                    lot_size = besoin_net
                
                planned_recs[i] = lot_size
                
                # Planifier le lancement avec le lead time
                if i - lead_time >= 0:
                    planned_rels[i - lead_time] = lot_size
            
            # Mettre à jour le stock prévisionnel
            projected_oh[i] = stock_disponible + planned_recs[i] - gross_reqs[i]
        
        # Stocker les lancements planifiés pour les composants enfants
        self.planned_releases[component_id] = planned_rels
        
        # Mettre à jour l'arbre
        tree.item(gross_req_id, values=gross_reqs)
        tree.item(scheduled_receipts_id, values=scheduled_receipts)
        tree.item(projected_on_hand_id, values=projected_oh)
        tree.item(net_req_id, values=[max(0, x) for x in net_reqs])
        tree.item(planned_receipts_id, values=planned_recs)
        tree.item(planned_releases_id, values=planned_rels)
      
     
    
    def calculate_eoq(self):
        """Calculer l'EOQ pour le composant sélectionné"""
        component_name = self.eoq_component_var.get()
        
        if not component_name:
            messagebox.showwarning("Avertissement", "Veuillez sélectionner un composant.")
            return
        
        # Récupérer les paramètres
        D = self.annual_demand_var.get()  # Demande annuelle
        S = self.ordering_cost_var.get()  # Coût de commande
        H = self.holding_cost_var.get()   # Coût de possession
        working_days = self.working_days_var.get()  # Jours ouvrés par an
        
        # Calculer EOQ
        Q_star = math.sqrt((2 * D * S) / H)
        num_orders = D / Q_star
        days_between_orders = working_days / num_orders
        
        # Afficher les résultats
        self.eoq_result_text.delete(1.0, tk.END)
        result_text = f"""Résultats EOQ pour {component_name}:
        
Quantité de commande optimale (EOQ): {Q_star:.2f} unités
Nombre de commandes par an: {num_orders:.2f} commandes
Intervalle entre commandes: {days_between_orders:.2f} jours ouvrés
        
Coût total annuel: ${(D * self.unit_cost_var.get() + (D / Q_star) * S + (Q_star / 2) * H):.2f}
  - Coût d'achat: ${D * self.unit_cost_var.get():.2f}
  - Coût de commande: ${(D / Q_star) * S:.2f}
  - Coût de possession: ${(Q_star / 2) * H:.2f}
        """
        self.eoq_result_text.insert(tk.END, result_text)
        
        # Tracer le graphique EOQ
        self.plot_eoq_graph(D, S, H)
        
    def plot_eoq_graph(self, D, S, H):
        """Tracer le graphique EOQ"""
        self.ax.clear()
        
        # Générer les données
        q_values = np.linspace(10, 5000, 100)
        ordering_cost = [(D / q) * S for q in q_values]
        holding_cost = [(q / 2) * H for q in q_values]
        total_cost = [ordering_cost[i] + holding_cost[i] for i in range(len(q_values))]
        
        # Calculer EOQ
        q_star = math.sqrt((2 * D * S) / H)
        total_cost_star = (D / q_star) * S + (q_star / 2) * H
        
        # Tracer les courbes
        self.ax.plot(q_values, ordering_cost, label='Coût de commande', color='blue')
        self.ax.plot(q_values, holding_cost, label='Coût de possession', color='green')
        self.ax.plot(q_values, total_cost, label='Coût total', color='red', linewidth=2)
        
        # Marquer le point EOQ
        self.ax.plot([q_star], [total_cost_star], 'ro', markersize=10)
        self.ax.annotate(f'EOQ = {q_star:.2f}', (q_star, total_cost_star), 
                      xytext=(q_star + 200, total_cost_star + 200),
                      arrowprops=dict(facecolor='black', shrink=0.05, width=1.5))
        
        # Configuration du graphique
        self.ax.set_xlabel('Quantité de commande (Q)')
        self.ax.set_ylabel('Coût ($)')
        self.ax.set_title('Analyse EOQ')
        self.ax.legend(loc='best')
        self.ax.grid(True)
        
        # Mettre à jour le canvas
        self.canvas.draw()
    
    def generate_report(self):
        """Générer le rapport sélectionné"""
        report_type = self.report_type_var.get()
        output_format = self.output_format_var.get()
        
        # Effacer le Treeview existant
        for col in self.report_tree["columns"]:
            self.report_tree.heading(col, text="")
        
        for item in self.report_tree.get_children():
            self.report_tree.delete(item)
        
        # Définir les colonnes en fonction du type de rapport
        if report_type == "Inventaire":
            self.report_tree["columns"] = ("name", "on_hand", "min_level", "max_level", "lead_time", "lot_size")
            self.report_tree.heading("#0", text="ID")
            self.report_tree.heading("name", text="Nom")
            self.report_tree.heading("on_hand", text="Stock")
            self.report_tree.heading("min_level", text="Min")
            self.report_tree.heading("max_level", text="Max")
            self.report_tree.heading("lead_time", text="Délai")
            self.report_tree.heading("lot_size", text="Lot")
            
            # Récupérer les données d'inventaire
            self.cursor.execute("SELECT id, name, on_hand, 0, 0, lead_time, lot_size FROM items")
            for row in self.cursor.fetchall():
                item_id, name, on_hand, min_level, max_level, lead_time, lot_size = row
                self.report_tree.insert("", "end", text=item_id, 
                                       values=(name, on_hand, min_level, max_level, lead_time, lot_size))
            
        elif report_type == "Planification des commandes":
            self.report_tree["columns"] = ("name", "qty", "date", "supplier", "status")
            self.report_tree.heading("#0", text="ID")
            self.report_tree.heading("name", text="Composant")
            self.report_tree.heading("qty", text="Quantité")
            self.report_tree.heading("date", text="Date")
            self.report_tree.heading("supplier", text="Fournisseur")
            self.report_tree.heading("status", text="Statut")
            
            # Dans une vraie application, vous récupéreriez les données réelles
            # Pour l'exemple, nous allons générer des données fictives
            self.report_tree.insert("", "end", text="1", values=("Trucks", "500", "2025-04-01", "Supplier A", "En cours"))
            self.report_tree.insert("", "end", text="2", values=("Wheels", "2000", "2025-03-22", "Supplier B", "En cours"))
            self.report_tree.insert("", "end", text="3", values=("Screws", "5000", "2025-03-15", "Supplier C", "Reçu"))
            
        elif report_type == "Besoins nets":
            self.report_tree["columns"] = ("name", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8")
            self.report_tree.heading("#0", text="ID")
            self.report_tree.heading("name", text="Composant")
            for i in range(1, 9):
                self.report_tree.heading(f"p{i}", text=f"P{i}")
            
            # Dans une vraie application, vous récupéreriez les données réelles
            # Pour l'exemple, nous allons générer des données fictives
            self.report_tree.insert("", "end", text="1", values=("Skateboard", "0", "0", "0", "0", "600", "0", "0", "0"))
            self.report_tree.insert("", "end", text="2", values=("Board", "0", "0", "0", "600", "0", "0", "0", "0"))
            self.report_tree.insert("", "end", text="3", values=("Trucks", "0", "0", "1200", "0", "0", "0", "0", "0"))
            
        elif report_type == "Résumé EOQ":
            self.report_tree["columns"] = ("name", "annual_demand", "eoq", "num_orders", "days_between")
            self.report_tree.heading("#0", text="ID")
            self.report_tree.heading("name", text="Composant")
            self.report_tree.heading("annual_demand", text="Demande annuelle")
            self.report_tree.heading("eoq", text="EOQ")
            self.report_tree.heading("num_orders", text="Commandes/an")
            self.report_tree.heading("days_between", text="Jours entre commandes")
            
            # Récupérer les composants achetés
            self.cursor.execute("SELECT id, name FROM items WHERE is_purchased = 1")
            purchased_items = self.cursor.fetchall()
            
            # Pour chaque composant, calculer l'EOQ
            for item_id, name in purchased_items:
                # Dans une vraie application, vous utiliseriez les données réelles
                # Pour l'exemple, nous allons utiliser les mêmes valeurs pour tous
                D = 60000  # Demande annuelle
                S = 65     # Coût de commande
                H = 5      # Coût de possession
                working_days = 200  # Jours ouvrés par an
                
                Q_star = round(math.sqrt((2 * D * S) / H), 2)
                num_orders = round(D / Q_star, 2)
                days_between_orders = round(working_days / num_orders, 2)
                # Insert data into the report tree
                self.report_tree.insert("", "end", text=item_id, 
                                       values=(name, D, Q_star, num_orders, days_between_orders))
        # Handle output format
        if output_format == "CSV":
            self.export_to_csv(report_type)
        elif output_format == "Excel":
            self.export_to_excel(report_type)
        messagebox.showinfo("Information", f"Rapport {report_type} généré avec succès!")

    def export_to_csv(self, report_type):
        """Export the current report to a CSV file"""
        filename = f"{report_type}_report.csv"
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            # Write headers
            headers = ["ID"] + [self.report_tree.heading(col)["text"] for col in self.report_tree["columns"]]
            writer.writerow(headers)
            # Write data
            for item in self.report_tree.get_children():
                row = [self.report_tree.item(item)["text"]] + list(self.report_tree.item(item)["values"])
                writer.writerow(row)
        messagebox.showinfo("Information", f"Rapport exporté en CSV: {filename}")

    def export_to_excel(self, report_type):
        """Export the current report to an Excel file"""
        filename = f"{report_type}_report.xlsx"
        df = pd.DataFrame(columns=["ID"] + [self.report_tree.heading(col)["text"] for col in self.report_tree["columns"]])
        for item in self.report_tree.get_children():
            row = [self.report_tree.item(item)["text"]] + list(self.report_tree.item(item)["values"])
            df.loc[len(df)] = row
        df.to_excel(filename, index=False)
        messagebox.showinfo("Information", f"Rapport exporté en Excel: {filename}")
    def on_close(self):
        # Perform any necessary cleanup here
        if hasattr(self, 'conn'):
            self.conn.close()  # Close the database connection
        # Destroy the window
        self.root.destroy()

    def run(self):
        """Run the application"""
        self.root.mainloop()

class LoginApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Connexion - Système MRP")
        self.root.geometry("400x400")
        self.root.configure(bg="#f0f0f0")

        # Initialisation des widgets
        self.setup_login_ui()

    def setup_login_ui(self):
        """Configuration de l'interface de login"""
        # Frame principal
        main_frame = ttk.Frame(self.root, padding=20)
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Titre
        title_label = ttk.Label(main_frame, text="CONNEXION", font=('Arial', 18, 'bold'))
        title_label.pack(pady=10)

        # Champ Nom d'utilisateur
        ttk.Label(main_frame, text="Nom d'utilisateur:").pack(pady=5)
        self.username_entry = ttk.Entry(main_frame, width=30)
        self.username_entry.pack(pady=5)

        # Champ Mot de passe
        ttk.Label(main_frame, text="Mot de passe:").pack(pady=5)
        self.password_entry = ttk.Entry(main_frame, width=30, show="*")
        self.password_entry.pack(pady=5)

        # Bouton de connexion
        login_button = ttk.Button(main_frame, text="Se connecter", command=self.validate_login)
        login_button.pack(pady=20)

        # Lien pour mot de passe oublié
        forgot_password_label = ttk.Label(main_frame, text="Mot de passe oublié?", foreground="blue", cursor="hand2")
        forgot_password_label.pack()
        forgot_password_label.bind("<Button-1>", lambda e: self.show_forgot_password())

        # Ajouter un bouton de sortie
        exit_button = ttk.Button(main_frame, text="Quitter", command=self.root.destroy)
        exit_button.pack(pady=10)

        def on_close(self):
            # Perform any necessary cleanup here
            if hasattr(self, 'conn'):
                self.conn.close()  # Close the database connection
            # Destroy the window
            self.root.destroy()



    def validate_login(self):
        """Validation des informations de connexion"""
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Simuler une base de données d'utilisateurs avec rôles
        valid_users = {
            "admin": {"password": "admin123", "role": "admin"},
            "user": {"password": "user123", "role": "user"},
            "1": {"password": "1", "role": "user"}
        }

        if username in valid_users and valid_users[username]["password"] == password:
            role = valid_users[username]["role"]
            self.open_main_application(role)
        else:
            messagebox.showerror("Erreur", "Identifiants incorrects")
    
    def open_main_application(self, role):
        """Ouvre l'application principale avec le bon rôle"""
        self.root.withdraw()
        main_app_window = tk.Toplevel(self.root)
        app = MRPSystem(main_app_window, is_admin=(role == "admin"))  # Passe le rôle
        main_app_window.protocol("WM_DELETE_WINDOW", lambda: self.on_main_app_close(main_app_window))
        
    def on_main_app_close(self, main_app_window):
        """Gère la fermeture de la fenêtre principale"""
        main_app_window.destroy()  # Ferme la fenêtre MRP
        self.root.deiconify()     # Réaffiche la fenêtre de login

    
    def show_forgot_password(self):
        """Afficher une fenêtre pour récupérer le mot de passe"""
        messagebox.showinfo("Mot de passe oublié", "Veuillez contacter l'administrateur pour réinitialiser votre mot de passe.")

if __name__ == "__main__":
    root = tk.Tk()
    login_app = LoginApp(root)
    root.mainloop()

C:\Users\johan\AppData\Local\Temp\ipykernel_24808\3368735804.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


MRPSystem window initialized
